In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from time import sleep
from random import randint

### Iterate each page for 1000 movie titles

In [4]:
headers = {"Accept-Language": "en-US,en;q=0.5"}

titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []


In [7]:
# Scraping loop code
pages = np.arange(1,1001,50)  #(start, stop, step)

for page in pages:
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt') # &ref_=adv_nxt added to the end of URL and it doesnt change when going to next page.
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    sleep(randint(2,10))  # 2-10 seconds
    
    for container in movie_div:
        name = container.h3.a.text
        titles.append(name)
        
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        runtime = container.p.find('span', class_='runtime').text if container.p.find('span', class_='runtime').text else '-'
        time.append(runtime)
        
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        nv = container.find_all('span', attrs={'name': 'nv'})
        
        vote = nv[0].text
        votes.append(vote)
        
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

In [8]:
movies = pd.DataFrame({
'movie': titles,
'year': years,
'timeMin': time,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'us_grossMillions': us_gross,
})

movies['year'] = movies['year'].str.extract('(\d+)').astype(int)

movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)

movies['votes'] = movies['votes'].str.replace(',', '').astype(int)

movies['metascore'] = movies['metascore'].str.extract('(\d+)')
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')

In [9]:
movies.to_csv('movies_2.csv')

In [11]:
movies.isnull().sum()

movie                 0
year                  0
timeMin               0
imdb                  0
metascore           162
votes                 0
us_grossMillions    180
dtype: int64

In [12]:
movies.metascore = movies.metascore.fillna('No Score Given')
movies.us_grossMillions = movies.us_grossMillions.fillna('')

In [14]:
movies.metascore.head()

0              75.0
1              71.0
2              73.0
3    No Score Given
4              78.0
Name: metascore, dtype: object

In [15]:
movies.us_grossMillions.head()

0          
1          
2    171.48
3          
4    858.37
Name: us_grossMillions, dtype: object